<a id='title'></a>
# Sources of variation in a measurement of depression for individual adolescents
Eric A. Miller, Giselle Sanchez, Alexandra Pryor, Janis H. Jenkins

<a id='index'></a>

# ---- Index ----

### [Data preprocessing](#Preprocess)
* Import libraries
* Figure plotting settings
* Load data
* Clean data


### [Figures](#FiguresHeading)
* [Figure 1](#Fig1)
* [Figure 2](#Fig2)
* [Figure 3](#Fig3)
* [Figure 4](#Fig4)

### [Supplemental Figures](#Supplementals):
* [Supplemental Figure 1](#FigS1)
* [Supplemental Figure 2](#FigS2)
* [Supplemental Figure 3](#FigS3)

### [Other calculations](#Various)
* Cronbach's alpha coefficient
* Intraclass correlation coefficient
* Uncertainty in internal state given total score
* GAD results not dependent on modified items
* Confirm PCA results using Scikit Learn

# ---- Data preprocessing ----
<a id='Preprocess'></a>

## Import libraries

In [ ]:
%matplotlib inline

# Python core
import os
import math
import datetime

# Scipy core
import numpy as np
from numpy import linalg as LA
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

# Other libraries
import dateutil
import seaborn as sns
import pingouin as pg
from sklearn.decomposition import PCA

## Figure plotting settings

In [ ]:
savefigs = False

# If saving figs, define your main output directory and inside that directory put empty
# subdirectories titled 'Fig1', 'Fig2', ..., 'FigS3' for each figure's panels to be saved
main_output_directory = ''
outdir = os.path.expanduser(main_output_directory)

# Variables for saving PDFs with matplotlib
plt.rcParams['pdf.fonttype'] = 42  # embed fonts so we can edit in Illustrator
plt.rcParams['font.sans-serif'] = ['Helvetica', 'Tahoma', 'Verdana']
resolution = 1000  # DPI

# Box plots
boxprops = dict(linewidth=1, color='k')
medianprops = dict(linewidth=0.9, color='k')

## Load data

In [ ]:
# Data available upon request
# phq = pd.read_csv('./data/phqA.csv')
# phq_adult = pd.read_csv('./data/phq9.csv')
# gad = pd.read_csv('./data/gad.csv')
# phq_dates = pd.read_csv('./data/phq_dates.csv')

## Clean data

In [ ]:
# Reindex the dataframes to start at 1 instead of 0, to match the student ID numbers
phq.index = np.arange(1, len(phq) + 1)
phq_adult.index = np.arange(1, len(phq_adult) + 1)
gad.index = np.arange(1, len(gad) + 1)
phq_dates.index = np.arange(1, len(phq_dates) + 1)

# 5, 44, and 47 dropped out of the study and contribute no data.
# 29, 36, and 42 left the school district; 29 and 36 completed the PHQ-9 (adult) before moving
excluded_adult = [5, 44, 47, 42]
excluded_child = [5, 44, 47, 29, 36, 42]
phq = phq.drop(excluded_child)
phq_adult = phq_adult.drop(excluded_adult)
gad = gad.drop(excluded_child)
phq_dates = phq_dates.drop(excluded_child)

# Four students never completed the adult PHQ form
no_phq_adult = [50, 51, 52, 53]
phq_adult = phq_adult.drop(no_phq_adult)
phq_dates = phq_dates.drop(no_phq_adult)

# Only keep PHQ dates for participants who completed both questionnaires, as these 
# will be used for evaluating change vs time elapsed.
phq_dates = phq_dates.dropna() 

# Cast numerical columns of the PHQ and GAD data to float
phq[phq.columns[1:]] = phq[phq.columns[1:]].astype('float64')
phq_adult[phq_adult.columns[1:]] = phq_adult[phq_adult.columns[1:]].astype('float64')
gad[gad.columns[1:]] = gad[gad.columns[1:]].astype('float64')

# Replace user-defined '99' code with NaN
phq = phq.replace(99, np.nan)
phq_adult = phq_adult.replace(99, np.nan)
gad = gad.replace(99, np.nan)

# Cast date strings to date objects (allows operations like subtraction of dates)
phq_dates['Adult PHQ Date'] = phq_dates['Adult PHQ Date'].apply(lambda d: dateutil.parser.parse(d))
phq_dates['Child PHQ Date'] = phq_dates['Child PHQ Date'].apply(lambda d: dateutil.parser.parse(d))

# ----  Figures ----
<a id='FiguresHeading'></a>
[return to top](#title)

# Fig 1. PHQ total scores not normally distributed and correlated with GAD
<a id='Fig1'></a>
[return to top](#title)

## a&b. Total score distributions and tests for normality

In [ ]:
# Histogram bins
phqbins = np.arange(0, 22, 2) # use same bins for both PHQ scales
gadbins = np.arange(0, 33, 3)

# Weights for converting histograms into frequency distributions
phqweights = np.ones_like(phq['PHQTOTAL'])/len(phq['PHQTOTAL'])
phq_adult_weights = np.ones_like(phq_adult['PHQTOTAL_Adult'])/len(phq_adult['PHQTOTAL_Adult'])
gadweights = np.ones_like(gad['GADRAW'])/len(gad['GADRAW'])


# PHQ-A ("Child & Adolescent" form)
plt.figure()
ax = plt.subplot(111)
plt.hist(phq['PHQTOTAL'], bins=phqbins, weights=phqweights, color='white', edgecolor='black')
plt.xlabel('PHQ-A Total Score', fontsize=14)
plt.ylabel('Fraction of students', fontsize=14)
plt.xticks([0, 4, 8, 12, 16, 20])
plt.yticks([0, 0.1, 0.2, 0.3])
plt.ylim([0, 0.35])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
W, p = stats.shapiro(phq['PHQTOTAL'])
print("Shapiro-Wilk Test for PHQ-A (Adolescent PHQ-9) Total: %f" % p)
if savefigs: 
    plt.savefig(fname=outdir+'Fig1/phqA_hist.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

# PHQ-9 ("Adult" form)
plt.figure()
ax = plt.subplot(111)
plt.hist(phq_adult['PHQTOTAL_Adult'], bins=phqbins, weights=phq_adult_weights, color='white', edgecolor='black')
plt.xlabel('PHQ-9 Total Score', fontsize=14)
plt.ylabel('Fraction of students', fontsize=14)
plt.xticks([0, 4, 8, 12, 16, 20])
plt.yticks([0, 0.1, 0.2, 0.3])
plt.ylim([0, 0.35])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
W, p = stats.shapiro(phq_adult['PHQTOTAL_Adult'])
print("Shapiro-Wilk Test for PHQ-9 (Adult) Total: %f" % p)
if savefigs:
    plt.savefig(fname=outdir+'Fig1/phq9_hist.pdf', format='pdf', dpi=resolution, bbox_inches="tight")


# GAD
plt.figure()
ax = plt.subplot(111)
plt.hist(gad['GADRAW'], bins=gadbins, weights=gadweights, color='white', edgecolor='black')
plt.xlabel('GAD Total Score', fontsize=14)
plt.ylabel('Fraction of students', fontsize=14)
# plt.xticks([0, 0.4, 0.8, 1.2, 1.6, 2.0])
plt.yticks([0, 0.1, 0.2, 0.3, 0.4])
plt.ylim([0, 0.4])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
W, p = stats.shapiro(gad['GADRAW'])
print("Shapiro-Wilk Test for GAD Total (Average): %f" % p)
if savefigs:
    plt.savefig(fname=outdir+'Fig1/gad_hist.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

## c. Correlation of PHQ and GAD

In [ ]:
df = pd.merge(phq, gad, on='ID')
plt.figure()
ax = plt.subplot(111)
plt.scatter(df['GADRAW'], df['PHQTOTAL'], color='k', alpha=0.6)
plt.xlabel('GAD-A Total Score', fontsize=14)
plt.ylabel('PHQ-A Total Score', fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 4, 8, 12, 16, 20])
r, p = stats.pearsonr(df['GADRAW'], df['PHQTOTAL'])
print("PHQ-A R-squared = %f" % r**2)
if savefigs:
    plt.savefig(fname=outdir+'Fig1/phqA_vs_gad.pdf', format='pdf', dpi=resolution, bbox_inches="tight")


df = pd.merge(phq_adult, gad, on='ID')    
plt.figure()
ax = plt.subplot(111)
plt.scatter(df['GADRAW'], df['PHQTOTAL_Adult'], color='k', alpha=0.6)
plt.xlabel('GAD-A Total Score', fontsize=14)
plt.ylabel('PHQ-9 Total Score', fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 4, 8, 12, 16, 20])
r, p = stats.pearsonr(df['GADRAW'], df['PHQTOTAL_Adult'])
print("PHQ-9 R-squared = %f" % r**2)
if savefigs:
    plt.savefig(fname=outdir+'Fig1/phq9_vs_gad.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

# Fig 2. Changes between the PHQ-9 and PHQ-A for individual subjects.
<a id='Fig2'></a>
[return to top](#title)

## Show vectors of PHQ change for each individual student

In [ ]:
df = pd.merge(phq, phq_adult, on='ID')
df['diffs'] =  np.array(df['PHQTOTAL'] - df['PHQTOTAL_Adult'])
df['avg'] = np.array((df['PHQTOTAL'] + df['PHQTOTAL_Adult']) / 2)

df_sort = df.sort_values(by='avg', ascending=True)
plt.figure(figsize=(17,7))
ax = plt.subplot(111)
plt.scatter(range(df_sort.shape[0]), df_sort['PHQTOTAL_Adult'], marker='o', color='k')
plt.scatter(range(df_sort.shape[0]), df_sort['PHQTOTAL'], marker='o', color='k')
ax.axhline(y=4, linestyle='--', color='grey')
ax.axhline(y=9, linestyle='--', color='grey')
ax.axhline(y=14, linestyle='--', color='grey')
plt.ylabel('PHQ Total', fontsize=12)
plt.yticks([0, 4, 9, 14, 19])
plt.xlabel('Participants', fontsize=12)
plt.xticks([])
for i in range(df_sort.shape[0]):
    row = df_sort.iloc[i, :]
    plt.arrow(x=i, y=row['PHQTOTAL_Adult'], dx=0, dy=row['diffs'], 
             length_includes_head=True, color='grey', facecolor='grey', head_width=0.8, head_length=0.7)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
if savefigs:
    plt.savefig(fname=outdir+'Fig2/all_diffs.pdf', format='pdf', dpi=resolution, bbox_inches="tight")


## Distribution of PHQ changes across students
Check for normality and for a consistent bias in direction of change.

In [ ]:
df = pd.merge(phq, phq_adult, on='ID')
df['diffs'] =  np.array(df['PHQTOTAL'] - df['PHQTOTAL_Adult'])
weights = np.ones_like(df['diffs'])/len(df['diffs'])

plt.figure(figsize=(8, 5))
ax = plt.subplot(111)
plt.hist(df['diffs'], weights=weights, bins=np.arange(-10, 12, 2), color='white', edgecolor='k')
plt.xlabel('Difference in PHQ', fontsize=12)
plt.ylabel('Fraction of students', fontsize=12)
plt.xticks([-10, -5, 0, 5, 10])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

print("One sample t-test vs 0:")
print(stats.ttest_1samp(df['diffs'], 0))

s, p = stats.shapiro(df['diffs'])
print('\nShapiro-Wilk test for normality: p = %f' % p)

if savefigs:
    plt.savefig(fname=outdir+'Fig2/diff_histogram.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

### Differences in PHQ-9 and PHQ-A total score vs. the time elapsed between measurements.
12 / 43 of the subjects who completed both the PHQ-9 and the PHQ-A had missing data for the date they completed the PHQ-A, so we were unable to calculate the elapsed time for these subjects. For the remaining 31 subjects, we were able to test whether the absolute difference depended on the elapsed time between the tests.

In [ ]:
df = pd.merge(phq, phq_adult, on='ID')
df['diffs'] =  np.array(df['PHQTOTAL'] - df['PHQTOTAL_Adult'])
diffs = list(phq_dates['Child PHQ Date'] - phq_dates['Adult PHQ Date'])
phq_dates['date_diffs'] = [d.days for d in diffs]
df2 = pd.merge(df, phq_dates, on='ID')

plt.figure(figsize=(8, 5))
ax = plt.subplot(111)
plt.scatter(df2['date_diffs'], np.abs(df2['diffs']), color='k', alpha=0.6)
plt.xlabel('Time elapsed (days)', fontsize=12)
plt.ylabel('Absolute value difference', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
r, p = stats.pearsonr(df2['date_diffs'], np.abs(df2['diffs']))
print("\nAbsolute value difference:")
print("p = %f, R = %f" % (p, r))
print("R-squared = %f" % r**2)
if savefigs:
    plt.savefig(fname=outdir+'Fig2/abs_diff_vs_time.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

### Bland-Altman plot
95% limits of agreement derived from this paper:
https://journals.sagepub.com/doi/pdf/10.1177/096228029900800204

In [ ]:
df = pd.merge(phq, phq_adult, on='ID')
df['diffs'] =  np.array(df['PHQTOTAL'] - df['PHQTOTAL_Adult'])
df['avg'] = np.array((df['PHQTOTAL'] + df['PHQTOTAL_Adult']) / 2)

plt.figure()
ax = plt.subplot(111)
plt.scatter(df['avg'], df['diffs'], color='k', alpha=0.6)
plt.xticks([0, 5, 10, 15])
plt.yticks([-15, -10, -5, 0, 5, 10, 15])
plt.ylabel('Difference in PHQ total', fontsize=12)
plt.xlabel('Average PHQ total', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

overall_mu = np.mean(df['diffs'])
overall_std = np.std(df['diffs'])
print("Overall mean: %f +/- %f" % (overall_mu, overall_std))


# Compute 95% limits of agreement
# --------
# 1. Regress the differences vs the averages to estimate D_hat
# D_hat = b1 + m1 * A
m1, b1, r_value, p_value, std_err = stats.linregress(df['avg'], df['diffs']) 
xs = np.linspace(0, np.max(df['avg']), 100)
ys = [m1 * x + b1 for x in xs]
plt.plot(xs, ys, linestyle='--', color='grey', alpha=0.7)
print("Overall slope: %f +/- %f" % (m1, std_err))

# 2. Regress the residuals of D_hat vs. the average to estimate R_hat
# R_hat = b2 + m2*A
res = []
for i in range(df.shape[0]):
    a = df['avg'][i]
    d_hat = m1 * a + b1
    d = df['diffs'][i]
    res.append(np.abs(d - d_hat))
m2, b2, r_value, p_value, std_err = stats.linregress(df['avg'], res)

# 3. Estimate the 95% limits of agreement D_hat +/- 2.46 * R_hat
ys2_upper = [(m1 * x + b1) + 2.46 * (m2 * x + b2) for x in xs]
ys2_lower = [(m1 * x + b1) - 2.46 * (m2 * x + b2) for x in xs]
plt.plot(xs, ys2_upper, linestyle='--', color='grey')
plt.plot(xs, ys2_lower, linestyle='--', color='grey')
if savefigs:
    plt.savefig(fname=outdir+'Fig2/bland_altman.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

# Fig 3. PHQ-9 and PHQ-A item responses show multidimensional structure.
<a id='Fig3'></a>
[return to top](#title)

### Graphical analysis of item response structure
This is a graphical approach to observe patterns in item responses across students. On the y-axis, we order students from highest to lowest total score, and on the x-axis we rank questions from highest to lowest total score. Thus the top left is where we should find the most high scores, and the bottom right is where we should see the most low scores. Another thing to look at is whether the ordering of students and questions are maintained between the two forms. Here, for example, we can see that far fewer high scores were reported for the "Movement" question in the Child form, compared to the adult form.

In [ ]:
# Adult form
phq_sort = phq_adult.iloc[:, :-2].sort_values(by='PHQTOTAL_Adult', ascending=False)
question_order = phq_sort.iloc[:, 1:-1].sum().sort_values(ascending=False).index.values
plt.figure(figsize=(7, 12))
ax = plt.subplot(111)
hm = sns.heatmap(phq_sort[question_order], cmap='Greys', cbar=False)
xlabels = [q.split("_")[0] for q in question_order]
hm.set_xticklabels(xlabels, rotation=30)
plt.title('PHQ-9', fontsize=12)
plt.xlabel('Item', fontsize=12)
plt.ylabel('Participant', fontsize=12)
plt.yticks([])
if savefigs:
    plt.savefig(fname=outdir+'Fig3/heatplot_phq9.pdf', format='pdf', dpi=resolution, bbox_inches="tight")


# Child form
phq_sort = phq.iloc[:, :-1].sort_values(by='PHQTOTAL', ascending=False)
question_order = phq_sort.iloc[:, 1:-1].sum().sort_values(ascending=False).index.values
plt.figure(figsize=(7, 12))
ax = plt.subplot(111)
hm = sns.heatmap(phq_sort[question_order], cmap="Greys", cbar=False)
hm.set_xticklabels(question_order, rotation=30)
plt.title('PHQ-A', fontsize=12)
plt.xlabel('Item', fontsize=12)
plt.ylabel('Participant', fontsize=12)
plt.yticks([])
if savefigs:
    plt.savefig(fname=outdir+'Fig3/heatplot_phqA.pdf', format='pdf', dpi=resolution, bbox_inches="tight")


## Principal components analysis (PCA) on the PHQ items
Standard PCA on the mean-subtracted item responses. We dropped 2 students in PHQ-A form and 1 student for PHQ-9 form due to missing values. These results were confirmed using the Scikit Learn library; see bottom of this notebook, in "Other" analyses. 

In [ ]:
def pca(A):
    cov = np.cov(A.T)
    w, v = LA.eig(cov.astype(float))
    vs = []  # eigenvectors
    for i in range(len(w)):
        vs.append(v[:, i])
    sorted_pairs = sorted(zip(w, vs), key=lambda x: x[0], reverse=True)
    w = [pair[0] for pair in sorted_pairs]
    v = [pair[1] for pair in sorted_pairs] 
    return w, v


# Create PHQ matrices that just have the item scores, with no metadata
# and ensure that the columns are ordered identically for the two tests
phq_clean = phq.iloc[:, 1:-2]
phq_adult_clean = phq_adult.iloc[:, 1:-3]
cols1 = list(phq_clean.columns)
cols2 = [c + "_Adult" for c in cols1]
phq_adult_clean = phq_adult_clean.loc[:, cols2]

# PCA on PHQ-A
A = phq_clean.copy()
A = A.dropna()  # 2 students dropped for missing val
# Subtract the mean of each column
for i, p in enumerate(A.columns):
    A.loc[:, p] = A[p] - A[p].mean()
A = np.array(A)
w1, v1 = pca(A)
sum1 = np.sum(w1)
varexp1 = [w/sum1 for w in w1]

# PCA on PHQ-9
A = phq_adult_clean.copy()
A = A.dropna()  # 1 student dropped for missing val
# Subtract the mean of each column
for i, p in enumerate(A.columns):
    A.loc[:, p] = A[p] - A[p].mean()
A = np.array(A)
w2, v2 = pca(A)
sum2 = np.sum(w2)
varexp2 = [w/sum2 for w in w2]

# Save PCs to excel tables
pcsA = np.array(v1)
pcs9 = np.array(v2)
df_A = pd.DataFrame(pcsA, index=np.arange(1, 10), columns=[cols1]).round(3)
df_9 = pd.DataFrame(pcs9, index=np.arange(1, 10), columns=[cols1]).round(3)
varexp = np.array([varexp1, varexp2]).T.round(2)
df_pctvar = pd.DataFrame(varexp, index=np.arange(1, 10), columns=['PHQ-A', 'PHQ-9'])
if savefigs:
    df_A.to_excel("./phqA_pcs.xlsx")
    df_9.to_excel("./phq9_pcs.xlsx") 
    df_pctvar.to_excel("./pctvar.xlsx")

## Analyze the scree plot and the coefficients of the first two PCs

In [ ]:
print("PHQ-A")
print("PC1 explains %f percent of variance" % varexp1[0])
print("PC2 explains %f percent of variance" % varexp1[1])

print("\nPHQ-9")
print("PC1 explains %f percent of variance" % varexp2[0])
print("PC2 explains %f percent of variance" % varexp2[1])


# Scree plot
plt.figure()
ax = plt.subplot(111)
plt.plot(range(len(varexp1)), varexp1, marker='o', color='k', label='PHQ-A')
plt.plot(range(len(varexp2)), varexp2, marker='^', color='k', linestyle='--', label='PHQ-9')
plt.ylabel("Fraction of variance explained", fontsize=12)
plt.xlabel('Principal component', fontsize=12)
plt.xticks(range(0, 9), range(1, 10))
plt.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
if savefigs:
    plt.savefig(fname=outdir+'Fig3/scree.pdf', format='pdf', dpi=resolution, bbox_inches="tight")



# PC1
cols = phq_clean.columns
plt.figure(figsize=(7, 7*0.55))
ax = plt.subplot(111)
plt.plot(v1[0], color='k', marker='o', label='PHQ-A')
plt.plot(v2[0], color='k', marker='^', linestyle='--', label='PHQ-9')
ax.axhline(y=0, linestyle='--', color='grey')
plt.xticks(range(len(v1[0])), cols, rotation=25)
plt.xlabel('Item', fontsize=12)
plt.ylabel('Coefficient', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks(np.arange(-0.2, 0.6, 0.2))
# plt.ylim([-0.8, 0.8])
if savefigs:
    plt.savefig(fname=outdir+'Fig3/pc1_coef.pdf', format='pdf', dpi=resolution, bbox_inches="tight")


# PC2
plt.figure(figsize=(7, 7*0.55))
ax = plt.subplot(111)
plt.plot(-v1[1], color='k', marker='o', label='PHQ-A')
plt.plot(v2[1], color='k', marker='^', linestyle='--', label='PHQ-9')
ax.axhline(y=0, linestyle='--', color='grey')
plt.xticks(range(len(v1[0])), cols, rotation=25)
plt.xlabel('Item', fontsize=12)
plt.ylabel('Coefficient', fontsize=12)
plt.yticks(np.arange(-0.8, 1.0, 0.4))
plt.ylim([-0.8, 0.8])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
if savefigs:
    plt.savefig(fname=outdir+'Fig3/pc2_coef.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

# Fig 5. Internal inconsistency in PHQ item responses for individual adolescents.
<a id='Fig5'></a>
[return to top](#title)

## Compute measures of internal inconsistency for each student
See Materials and Methods for definitions of these quantities:
* points of disagreement
* separate items disagree on
* total internal change (in either direction)
* percent disagreement



### Create a new dataframe (df_disagree) that contains the inconsistency metrics for each participant

In [ ]:
colnames = np.array(phq.columns[1:-2]) # ensure we compare same items from both questionnaires
df_disagree = pd.DataFrame(columns=['ID', 'net_diff', 'pts_disagree', 'items_disagree', 'total_change', 'pct_disagree'])
df = pd.merge(phq, phq_adult, on='ID')
df['diffs'] =  np.array(df['PHQTOTAL'] - df['PHQTOTAL_Adult'])
idx = 0
for sid in df['ID']:
    row = df[df['ID']==sid]
    net_diff = np.round(row['diffs'].values[0]) # net diff in total score
    # Now we compute the new quantitites
    pts_disagree = 0
    items_disagree = 0
    total_change = 0
    for col in colnames:    
        col_adult = col + "_Adult"
        item_diff = (row[col] - row[col_adult]).values[0]  # difference just on this item
        if np.isnan(item_diff) or item_diff == 0: 
            continue
        else:
            total_change += np.abs(item_diff)           # update the total internal change
            if (((net_diff > 0) and (item_diff < 0)) or ((net_diff < 0) and (item_diff > 0))):
                # PD is defined as the item change in opposite direction from net change
                pts_disagree += np.abs(item_diff)
                items_disagree += 1
            elif (net_diff == 0):
                # If 0 net change, PD is defined as 1/2 total item changes
                pts_disagree += 0.5 * np.abs(item_diff)
                items_disagree += 1
    # Compute percent disagreement
    if total_change != 0:
        pct_disagree = 100 * pts_disagree / total_change
    else:
        pct_disagree = 0
    df_disagree.loc[idx] = [sid, np.abs(net_diff), pts_disagree, items_disagree, total_change, pct_disagree]
    idx += 1

### Helper function for plotting individual participnts' data

In [ ]:
# Running this function requires the df_disagree dataframe to have already been computed (preceding cell)
# it also requires the base dataframes (phq and phq_adult) to have been computed. 
def plot_subject_items(sid):
    colnames = np.array(phq.columns[1:-2])
    df = pd.merge(phq, phq_adult, on='ID')
    df['diffs'] =  np.array(df['PHQTOTAL'] - df['PHQTOTAL_Adult'])
    net_change = df[df['ID']==sid]['diffs']
    pts_inconsistent = df_disagree[df_disagree['ID']==sid]['pts_disagree']
    # Get the item responses to the PHQ-9 (adult form) and PHQ-A (child form) for this participant
    adult_answers = []
    child_answers = []
    for col in colnames:
        col_adult = col + "_Adult"
        adult_answers.append(df[df['ID']==sid][col_adult].values[0])
        child_answers.append(df[df['ID']==sid][col].values[0])
    
    plt.figure(figsize=(7, 7*0.55))
    ax = plt.subplot(111)
    plt.title("Net change: %d, Disagreement: %d" % (net_change, pts_inconsistent), fontsize=12)
    plt.ylabel("Item response", fontsize=12)
    plt.xlabel("Item", fontsize=12)
    plt.xticks(range(len(colnames)), colnames, rotation=30)
    plt.yticks([0, 1, 2, 3])
    plt.ylim([-0.5, 3.5])
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    plt.scatter(range(len(colnames)), adult_answers, marker='o', linestyle="None", color='k')
    plt.scatter(range(len(colnames)), child_answers, marker='o', linestyle="None", color='k')
    # Draw the arrows from the PHQ-9 (adult) to the PHQ-A (child) response
    for i in range(len(colnames)):
        diff = child_answers[i] - adult_answers[i]
        plt.arrow(x=i, y=adult_answers[i], dx=0, dy=diff,
        length_includes_head=True, color='grey', facecolor='grey', head_width=0.25, head_length=0.2)
    if savefigs:
        plt.savefig(fname=outdir+'Fig4/individuals/%s.pdf' % sid, format='pdf', dpi=resolution, bbox_inches="tight")

## Fig. 4 a&b. Two individual participants

In [ ]:
plot_subject_items('ST35')
plot_subject_items('ST30')

## Fig. 4 c, d, & e.  Distributions of disagreement metrics
See Materials and Methods for definitions of these quantities:
* Total internal points of change
* Points of disagreement
* Percent disagreement

In [ ]:
total_change = df_disagree['total_change'].values
weights = np.ones_like(total_change)/len(total_change)
plt.figure(figsize=(6, 6))
ax = plt.subplot(111)
plt.hist(total_change, bins=np.arange(-0.5, 11.5, 1), weights=weights, rwidth=0.8, color='w', edgecolor='k')
plt.yticks([0, 0.05, 0.10, 0.15, 0.20])
plt.ylim([0, 0.23])
plt.xlabel('Total internal points of change', fontsize=12)
plt.ylabel('Fraction of participants', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
if savefigs:
    plt.savefig(fname=outdir+'Fig4/hist_internal_change.pdf', format='pdf', dpi=resolution, bbox_inches="tight")
print("Total points of change (P_T)")
print("Mean: %f, Standard dev: %f" % (np.mean(total_change), np.std(total_change)))

all_disagree = df_disagree['pts_disagree'].values
weights = np.ones_like(all_disagree)/len(all_disagree)
plt.figure(figsize=(6, 6))
ax = plt.subplot(111)
plt.hist(all_disagree, weights=weights, bins=[0, 1, 2, 3, 4, 5], rwidth=0.8, color='w', edgecolor='k')
plt.xticks([0.5, 1.5, 2.5, 3.5, 4.5], [0, 1, 2, 3, 4])
plt.yticks([0, 0.1, 0.2, 0.3, 0.4])
plt.ylim([0, 0.45])
plt.xlabel('Points of disagreement', fontsize=12)
plt.ylabel('Fraction of participants', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
if savefigs:
    plt.savefig(fname=outdir+'Fig4/hist_pts_disagree.pdf', format='pdf', dpi=resolution, bbox_inches="tight")
    
print("\nPoints of disagreement (P_D)")
print("Mean: %f, Standard dev: %f" % (np.mean(all_disagree), np.std(all_disagree)))

pct_disagree = df_disagree['pct_disagree']
pct_disagree = pct_disagree.dropna().values # NAs arise from
weights = np.ones_like(pct_disagree)/len(pct_disagree)
plt.figure(figsize=(6, 6))
ax = plt.subplot(111)
plt.yticks(np.arange(0, 21, 3))
plt.hist(pct_disagree, weights=weights, bins=[0, 10, 20, 30, 40, 50], color='w', edgecolor='k')
plt.yticks(np.arange(0, 0.5, 0.1))
plt.ylim([0, 0.45])
plt.xlabel('Percent disagreement', fontsize=12)
plt.ylabel('Fraction of participants', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
if savefigs:
    plt.savefig(fname=outdir+'Fig4/hist_percent_disagree.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

print("\nPercent disagreement (100 x P_D / P_T)")
print("Mean: %f, Median: %f, Standard dev: %f" % 
      (np.mean(pct_disagree), np.median(pct_disagree), np.std(pct_disagree)))    

delta = df_disagree[df_disagree['items_disagree'] > 0]  # subjects with at least 1 item of inconsistency
print('\n%d / %d subjects had at least 1 item of inconsistency' % (delta.shape[0], df.shape[0]))

item_pts = delta['pts_disagree'] - delta['items_disagree']
multi_delta = len(np.where(item_pts > 0)[0])
print('%d / %d subjects had an inconsistent item with >1 pt of change' % (multi_delta, delta.shape[0]))
print("So the majority were small inconsistencies on several items, rather than big inconsistencies on one item.")


# ---- Supplemental Figures ----
<a id='Supplementals'></a>
[return to top](#title)

# Supplemental Fig 1. Total scores after square root transformation
<a id='FigS1'></a>
[return to top](#title)

## a&b. Total score distributions and tests for normality

In [ ]:
# Weights for converting to frequency histograms
phqweights = np.ones_like(phq['PHQTOTAL'])/len(phq['PHQTOTAL'])
phq_adult_weights = np.ones_like(phq_adult['PHQTOTAL_Adult'])/len(phq_adult['PHQTOTAL_Adult'])
gadweights = np.ones_like(gad['GADRAW'])/len(gad['GADRAW'])

# Child & Adolescent PHQ -- Square root transformation
bins = np.arange(0, 6, 0.5)
plt.figure()
ax = plt.subplot(111)
sqrt_phq = phq['PHQTOTAL'].apply(lambda x: np.sqrt(x))
plt.hist(sqrt_phq, color='white', weights=phqweights, edgecolor='black')
plt.xlabel('Square root of PHQ-A total score', fontsize=14)
plt.ylabel('Fraction of students', fontsize=14)
plt.yticks([0, 0.05, 0.1, 0.15, 0.2, 0.25])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
W, p = stats.shapiro(sqrt_phq)
print("Shapiro-Wilk Test for SQUARE ROOT of PHQ-A Total: %f" % p)
if savefigs:
    plt.savefig(fname=outdir+'FigS1/sqrt_phqA_hist.pdf', format='pdf', dpi=resolution, bbox_inches="tight")


# PHQ-9 (Adult) -- Square root transformation
bins = np.arange(0, 6, 0.5)
plt.figure()
ax = plt.subplot(111)
sqrt_phq = phq_adult['PHQTOTAL_Adult'].apply(lambda x: np.sqrt(x))
plt.hist(sqrt_phq, color='white', weights=phq_adult_weights, edgecolor='black')
plt.xlabel('Square root of PHQ-9 total score', fontsize=14)
plt.ylabel('Fraction of students', fontsize=14)
plt.yticks([0, 0.05, 0.1, 0.15, 0.2, 0.25])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
W, p = stats.shapiro(sqrt_phq)
print("Shapiro-Wilk Test for SQUARE ROOT of PHQ-9 Total: %f" % p)
if savefigs:
    plt.savefig(fname=outdir+'FigS1/sqrt_phq9_hist.pdf', format='pdf', dpi=resolution, bbox_inches="tight")



# GAD -- Square root transformation
plt.figure()
ax = plt.subplot(111)
sqrt_gad = gad['GADRAW'].apply(lambda x: np.sqrt(x))
plt.hist(sqrt_gad, weights=gadweights, color='white', edgecolor='black')
plt.xlabel('Square root of GAD-A total score', fontsize=14)
plt.ylabel('Fraction of students', fontsize=14)
plt.yticks([0, 0.05, 0.1, 0.15, 0.2, 0.25])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
W, p = stats.shapiro(sqrt_gad)
print("Shapiro-Wilk Test for SQUARE ROOT of GAD Total (Average): %f" % p)
if savefigs:
    plt.savefig(fname=outdir+'FigS1/sqrt_gad_hist.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

## c. PHQ vs GAD linear correlation after square root transform

In [ ]:
df = pd.merge(phq, gad, on='ID')
df['PHQTOTAL'] = df['PHQTOTAL'].apply(lambda x: np.sqrt(x))
df['GADRAW'] = df['GADRAW'].apply(lambda x: np.sqrt(x))
plt.figure()
ax = plt.subplot(111)
plt.scatter(df['GADRAW'], df['PHQTOTAL'], color='k', alpha=0.7)
plt.xlabel('Square root of\nGAD-A Total Score', fontsize=14)
plt.ylabel('Square root of\nPHQ-A Total Score', fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
r, p = stats.pearsonr(df['GADRAW'], df['PHQTOTAL'])
print("PHQ-A: p = %f, R-squared = %f" % (p, r**2))
if savefigs:
    plt.savefig(fname=outdir+'FigS1/sqrt_phqA_vs_gad.pdf', format='pdf', dpi=resolution, bbox_inches="tight")


df = pd.merge(phq_adult, gad, on='ID')
df['PHQTOTAL_Adult'] = df['PHQTOTAL_Adult'].apply(lambda x: np.sqrt(x))
df['GADRAW'] = df['GADRAW'].apply(lambda x: np.sqrt(x))              
plt.figure()
ax = plt.subplot(111)
plt.scatter(df['GADRAW'], df['PHQTOTAL_Adult'], color='k', alpha=0.7)
plt.xlabel('Square root of\nGAD-A Total Score', fontsize=14)
plt.ylabel('Square root of\nPHQ-9 Total Score', fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
r, p = stats.pearsonr(df['GADRAW'], df['PHQTOTAL_Adult'])
print("PHQ-9: p = %f, R-squared = %f" % (p, r**2))
if savefigs:
    plt.savefig(fname=outdir+'FigS1/sqrt_phq9_vs_gad.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

# Supplemental Fig 2. Frequency distribution of items and response categories
<a id='FigS2'></a>
[return to top](#title)

In [ ]:
from textwrap import wrap

phqweights = np.ones_like(phq['PHQTOTAL'])/len(phq['PHQTOTAL'])
phq_adult_weights = np.ones_like(phq_adult['PHQTOTAL_Adult'])/len(phq_adult['PHQTOTAL_Adult'])
colnames = phq.columns[1:-2]
colors = plt.cm.jet(np.linspace(0, 1, len(colnames)))

df = phq.copy()
plt.figure()
ax = plt.subplot(111)
for i, colname in enumerate(colnames):
    col = phq[colname]
    valid_vals = col.dropna()
    weights = np.ones_like(valid_vals)/len(valid_vals)
    responses, bins = np.histogram(valid_vals, bins=[0,1,2,3,4], weights=weights)
    plt.plot(range(3), responses[1:], marker='o', label=colname, color=colors[i])
plt.legend(frameon=False, bbox_to_anchor=(1, 1.02))
plt.title('PHQ-A', fontsize=12)
plt.xticks([0, 1, 2], ['Several', 'More than\nHalf', 'Nearly every'])
plt.ylabel('Fraction of students', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 0.1, 0.2, 0.3, 0.4])
if savefigs:
    plt.savefig(fname=outdir+'FigS2/item_freq_phqA.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

df = phq_adult.copy()
plt.figure()
ax = plt.subplot(111)
for i, colname in enumerate(colnames):
    colname_adult = colname + "_Adult"
    col = phq_adult[colname_adult]
    valid_vals = col.dropna()
    weights = np.ones_like(valid_vals)/len(valid_vals)
    responses, bins = np.histogram(valid_vals, bins=[0,1,2,3,4], weights=weights)
    plt.plot(range(3), responses[1:], marker='o', label=colname, color=colors[i])
plt.title('PHQ-9', fontsize=12)
plt.xticks([0, 1, 2], ['Several', 'More than\nHalf', 'Nearly every'])
plt.ylabel('Fraction of subjects', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 0.1, 0.2, 0.3, 0.4])
if savefigs:
    plt.savefig(fname=outdir+'FigS2/item_freq_phq9.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

# Supplemental Fig 3. PHQ total scores by rater and response format
(a) Box plots of PHQ-9 and PHQ-A total scores, broken out by the three researchers.
(b) Box plots of PHQ-9 broken out by self-report vs out-loud.

<a id='FigS3'></a>
[return to top](#title)

In [ ]:
rater1 = ['ST1', 'ST3', 'ST14', 'ST15', 'ST16', 'ST17', 'ST20', 'ST21', 
          'ST24', 'ST25', 'ST26', 'ST27', 'ST28', 'ST35', 'ST37', 'ST29', 'ST36']
rater2 = ['ST12', 'ST13', 'ST19', 'ST22', 'ST31', 'ST32', 'ST38', 
          'ST41', 'ST48', 'ST49', 'ST50', 'ST51', 'ST52', 'ST53']
rater3 = ['ST2', 'ST4', 'ST6', 'ST7', 'ST8', 'ST9', 'ST10', 'ST11', 'ST18', 
          'ST23', 'ST30', 'ST33', 'ST34', 'ST39', 'ST40', 'ST43', 'ST45', 'ST46']
rater3_idx = [2, 4, 6, 7, 8, 9, 10, 11, 18, 23, 30, 33, 34, 39, 40, 43, 45, 46]

### (a) PHQ total score by different raters

In [ ]:
df = phq.copy()
r1 = df[df['ID'].isin(rater1)]
r2 = df[df['ID'].isin(rater2)]
r3 = df[df['ID'].isin(rater3)]
plt.figure(figsize=(7, 6))
ax = plt.subplot(111)
bplot = plt.boxplot([r1['PHQTOTAL'], r2['PHQTOTAL'], r3['PHQTOTAL']], 
                    patch_artist=True, boxprops=boxprops, medianprops=medianprops)
bplot['boxes'][0].set_facecolor('w')
bplot['boxes'][1].set_facecolor('w')
bplot['boxes'][2].set_facecolor('w')
plt.xticks([1, 2, 3], ['Rater 1', 'Rater 2', 'Rater 3'], fontsize=12)
plt.ylabel('PHQ-A Total Score', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 4, 8, 12, 16, 20])
if savefigs:
    plt.savefig(fname=outdir+'FigS3/box_raters_phqA.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

    
df = phq_adult.copy()
r1 = df[df['ID'].isin(rater1)]
r2 = df[df['ID'].isin(rater2)]
r3 = df[df['ID'].isin(rater3)]
plt.figure(figsize=(7, 6))
ax = plt.subplot(111)
bplot = plt.boxplot([r1['PHQTOTAL_Adult'], r2['PHQTOTAL_Adult'], r3['PHQTOTAL_Adult']], 
                    patch_artist=True, boxprops=boxprops, medianprops=medianprops)
bplot['boxes'][0].set_facecolor('w')
bplot['boxes'][1].set_facecolor('w')
bplot['boxes'][2].set_facecolor('w')
plt.xticks([1, 2, 3], ['Rater 1', 'Rater 2', 'Rater 3'], fontsize=12)
plt.ylabel('PHQ-9 Total Score', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 4, 8, 12, 16, 20])
if savefigs:
    plt.savefig(fname=outdir+'FigS3/box_raters_phq9.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

### (b) PHQ total scores by response format
Several students completed the PHQ-9 out-loud instead of self-report.

In [ ]:
outloud_adult_ids = ['ST2', 'ST6', 'ST7', 'ST9', 'ST13', 'ST18', 'ST30', 
                     'ST32', 'ST34', 'ST41', 'ST43', 'ST45', 'ST46']

df = phq_adult.copy()
outloud = df[df['ID'].isin(outloud_adult_ids)]
selfreport = df[~df['ID'].isin(outloud_adult_ids)]
plt.figure(figsize=(7, 6))
ax = plt.subplot(111)
bplot = plt.boxplot([outloud['PHQTOTAL_Adult'], selfreport['PHQTOTAL_Adult']], 
                     patch_artist=True, boxprops=boxprops, medianprops=medianprops)
bplot['boxes'][0].set_facecolor('w')
bplot['boxes'][1].set_facecolor('w')
plt.xticks([1, 2], ['Out loud', 'Self report'], fontsize=12)
plt.ylabel('PHQ-9 Total Score', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 4, 8, 12, 16, 20])
if savefigs:
    plt.savefig(fname=outdir+'FigS3/box_response_phq9.pdf', format='pdf', dpi=resolution, bbox_inches="tight")

print('\nPHQ-9:')
print("Out loud median (n=%d): %f" % (outloud.shape[0], np.median(outloud['PHQTOTAL_Adult'])))
print("Self report median (n=%d): %f" % (selfreport.shape[0], np.median(selfreport['PHQTOTAL_Adult'])))
U, p = stats.mannwhitneyu(outloud['PHQTOTAL_Adult'], selfreport['PHQTOTAL_Adult'], alternative='greater')
cles = U / (outloud.shape[0] * selfreport.shape[0])  # common language effect size
print("Mann-Whitney U = %f, p = %f" % (U, p))
print("Common Language Effect Size: %f" % cles)

# Various calculations not in figures
<a id='Various'></a>
[return to top](#title)

## Cronbach's Alpha as a measure of item interrelatedness
Cronbach's alpha represents a measure of interrelatedness of items, often described as "internal consistency", and is a lower bound on "reliability" as it is classically defined. Note that Cronbach's alpha does not imply unidimensionality, nor does it necessarily imply high covariances among individual items of the scale. To address the latter, we also compute the mean covariance for each item. These results were confirmed with SPSS.

In [ ]:
# Cronbach's Alpha
# -----------------


phq_clean = phq.iloc[:, 1:-2]  # just the item response cols
phq_adult_clean = phq_adult.iloc[:, 1:-3]
alpha_child = pg.cronbach_alpha(phq_clean, nan_policy='pairwise')
alpha_adult = pg.cronbach_alpha(phq_adult_clean, nan_policy='pairwise')

print("Cronbach's Alpha: PHQ-A")
print(alpha_child)
print("\nCronbach's Alpha: PHQ-9")
print(alpha_adult)


# PHQ-A: Compute average inter-item correlations (not including self-correlations)
corr_mat = phq_clean.corr(method='pearson')
all_corrs = []
for colname in corr_mat.columns:
    nonself = [c for c in corr_mat.columns if c != colname]
    corrs = corr_mat.loc[nonself, colname].values
    all_corrs.extend(corrs)
print("\nPHQ-A:")
print("Average (stdev) iteritem correlation = %f +/- %f" % (np.mean(all_corrs), np.std(all_corrs)))


# PHQ-9: Compute average inter-item correlations (not including self-correlations)
corr_mat = phq_adult_clean.corr(method='pearson')
all_corrs = []
for colname in corr_mat.columns:
    nonself = [c for c in corr_mat.columns if c != colname]
    corrs = corr_mat.loc[nonself, colname].values
    all_corrs.extend(corrs)
print("\nPHQ-9:")
print("Average (stdev) iteritem correlation = %f +/- %f" % (np.mean(all_corrs), np.std(all_corrs)))

## Intraclass Correlation Coefficient (ICC) as a measure of test-retest reliability
We report ICC2 which measures absolute agreement with random raters. Commenting the square root transformation will show the results prior to transformation. These results were confirmed with the output of SPSS.

In [ ]:
df = pd.merge(phq, phq_adult, on='ID')
df['PHQTOTAL'] = df['PHQTOTAL'].apply(lambda x: np.sqrt(x))
df['PHQTOTAL_Adult'] = df['PHQTOTAL_Adult'].apply(lambda x: np.sqrt(x))

ids = np.array(df['ID'])
ids_repeat = np.concatenate((ids, ids))
child = np.array(df['PHQTOTAL'])
adult = np.array(df['PHQTOTAL_Adult'])
both = np.concatenate((child, adult))
tests = ['child']*len(child) + ['adult']*len(adult)
df2 = pd.DataFrame(columns=['ID', 'SCORE', 'TEST'])
df2['ID'] = ids_repeat
df2['SCORE'] = both
df2['TEST'] = tests

print("ICC with absolute agreement (prior to square root transform)")
pg.intraclass_corr(data=df2, targets='ID', raters='TEST', ratings='SCORE')

## Uncertainty of internal state, given a total score 
The actual entropy would require computing the empirical probabilities of each microstate.

In [ ]:
import itertools
from scipy.stats import norm

sets = [[0, 1, 2, 3]]*9
omega = itertools.product(*sets)
sums = [np.sum(outcome) for outcome in omega]
weights = np.ones_like(sums)/len(sums)

plt.figure()
ax = plt.subplot(111)
plt.title('Number of ways of getting each total score')
plt.hist(sums, weights=weights, bins=np.arange(-0.5, 27.5, 1), color='w', edgecolor='k')
plt.ylabel('Fraction of internal states', fontsize=12)
plt.xlabel('Total score', fontsize=12)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
mean, std = norm.fit(sums)
x = np.linspace(0, 28, 100)
y = norm.pdf(x, mean, std)
plt.plot(x, y, color='grey')

## Uncertainty of changes in internal state between two total scores

In [ ]:
sets = [[0, 1, 2, 3]]*9
omega = itertools.product(*sets)
sums = [np.sum(outcome) for outcome in omega]

num_ways = {}
for s in sums:
    if s not in num_ways.keys():
        num_ways[s] = 1
    else:
        num_ways[s] += 1

num_change_ways = {}
for score1 in num_ways.keys():
    num_change_ways[score1] = {}
    for score2 in num_ways.keys():    
        if score2 == score1:
            continue
        num_change_ways[score1][score2] = num_ways[score1] * num_ways[score2]


xs = np.arange(0, 28, 1)
plt.figure()
plt.xlabel('End total score', fontsize=12)
plt.ylabel('Number of ways of change', fontsize=12)
for i in range(28):
    plt.plot(np.delete(xs, i), list(num_change_ways[i].values()), label=i)
plt.legend(title='Start total score', bbox_to_anchor=(1.05, 1))

## GAD results not dependent on modified items

In [ ]:
# GAD without the questions derived from #10
gad_sub = gad.drop(['ALC', 'WEED', 'MEDS', 'OBJCT', 'HELP', 'ACTVTY', 'RLIGON'], axis=1)
gsc = gad_sub.iloc[:, 1:-3]
gsc['gadraw'] = gsc.sum(axis=1)
gsc['ID'] = gad['ID']

plt.figure()
ax = plt.subplot(111)
plt.hist(gsc['gadraw'], color='white', edgecolor='black')
plt.xlabel('GAD-A total score\nnot including modified items', fontsize=14)
plt.ylabel('Number of students', fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
W, p = stats.shapiro(gsc['gadraw'])
print("Shapiro-Wilk test: %f" % p)

df = pd.merge(phq, gsc, on='ID')
plt.figure()
ax = plt.subplot(111)
plt.scatter(df['gadraw'], df['PHQTOTAL'], color='k')
plt.xlabel('GAD-A total score\nnot including modified items', fontsize=14)
plt.ylabel('PHQ-A total score', fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 4, 8, 12, 16, 20])
r, p = stats.pearsonr(df['gadraw'], df['PHQTOTAL'])
print("PHQ-A r-squared = %f" % r**2)

df = pd.merge(phq_adult, gsc, on='ID')                
plt.figure()
ax = plt.subplot(111)
plt.scatter(df['gadraw'], df['PHQTOTAL_Adult'], color='k')
plt.xlabel('GAD-A total score\nnot including modified items', fontsize=14)
plt.ylabel('PHQ-9 Total Score', fontsize=14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.yticks([0, 4, 8, 12, 16, 20])
r, p = stats.pearsonr(df['gadraw'], df['PHQTOTAL_Adult'])
print("PHQ-9 r-squared = %f" % r**2)



## Bland-Altman plot on square root PHQ total
Note that the "fan out" pattern could be largely or fully due to the fact that the majority of measurments were very small, which would necessarily produce a "squeezing" effect on the Bland-Altman plot.

In [ ]:
df = pd.merge(phq, phq_adult, on='ID')
df['PHQTOTAL'] = df['PHQTOTAL'].apply(lambda x: np.sqrt(x))
df['PHQTOTAL_Adult'] = df['PHQTOTAL_Adult'].apply(lambda x: np.sqrt(x))
df['diffs'] =  np.array(df['PHQTOTAL'] - df['PHQTOTAL_Adult'])
df['avg'] = np.array((df['PHQTOTAL'] + df['PHQTOTAL_Adult']) / 2)

plt.figure()
ax = plt.subplot(111)
plt.scatter(df['avg'], df['diffs'], color='k', alpha=0.6)
plt.xticks([0, 1, 2, 3, 4])
plt.yticks([-2, -1, 0, 1, 2])
plt.ylabel('Difference in $\sqrt{\mathrm{PHQ}}$', fontsize=12)
plt.xlabel('Average $\sqrt{\mathrm{PHQ}}$', fontsize=12)
mu = np.mean(df['diffs'])
sdev = np.std(df['diffs'])
ax.axhline(y=mu, linestyle='--', color='grey')
ax.axhline(y=mu+2*sdev, linestyle='--', color='grey')
ax.axhline(y=mu-2*sdev, linestyle='--', color='grey')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

## Confirm PCA results using scikit learn
We computed PCA by hand in Figure 3. Here we confirm the results using the Scikit Learn library.

In [ ]:
phq_clean = phq.iloc[:, 1:-2]
pca_child = PCA().fit(phq_clean.dropna())
print("Percent variance explained by PCs of PHQ-A responses:")
print(pca_child.explained_variance_ratio_)
print("\nAll PCs of PHQ-A responses:")
print(pd.DataFrame(pca_child.components_, index=np.arange(1,10,1), columns=phq_clean.columns))

phq_adult_clean = phq_adult.iloc[:, 1:-3]
pca_adult = PCA().fit(phq_adult_clean.dropna())
print("\nPercent variance explained by PCs of PHQ-9 responses:")
print(pca_adult.explained_variance_ratio_)
print("\nAll PCs of PHQ-9 responses:")
print(pd.DataFrame(pca_adult.components_, index=np.arange(1,10,1), columns=phq_adult_clean.columns))